# Notes on the Methods and Formulas

### Parameters
Parameters can be set in `configs.parameters.json`, for example:
```json
{
    "mc_horizon": 100,
    "mc_paths": 10000,
    "var_accounts": 1000,
    "var_percentile": 0.1,
    "accounts_min_borrow_eth": 0.2,
    "accounts_max_health": 2.0
}
```

#### The Graph (Compound-v2)
Used for current information on Compound. Total borrows, total supply, exchange rate, collateral factor and so on. 
Api at `src.the_graph_api.TheGraphAPI`. 

#### Coingecko (Historical Prices)
Use file: `data/coingecko_prices.csv`

Created by download script `download_historical_prices.py`. The script has a runtime of ~1h, because it sends about 600 requests to coingecko. The free API allows 10-50 requests per minute, so I use a sleep of 6 seconds to never send more than 10 requests per minute. 
Api at `src.coingecko_api.CoingeckoAPI`. 

Time window: `2022-11-21` to `2022-11-20`

Should be updated if gets too old compared to the timestamp of data from The Graph. 

### Volatility and Simulation of Trajectories
Calculated from the historical prices timeseries. Standard deviation of the log-returns. The log is used, because for the Monte Carlo method we model the price path by a Brownian Motion / Weiner Process, where the log-returns follow a Normal Distribution with $\mu=0$ and $\sigma=vol$. 

Implementation: `src.measures.calc_volatility`, `src.measures.monte_carlo_simulation`

### Maximum Insolvency for an Account

To estimate the maximum Insolvency, we use the 10 simulated szenarios of assets prices. 

`borrowValue` for each asset is calculated using the field `borrow_balance_underlying` of the account. 

`collateralValue` is the sum of `supply_balance_underlying` multiplied by their `collatoralFactor`s. 

Then the Insolvency is `insolvency = borrowValue - collateralValue`. We do this 10 times (for the 10 simulated szenarios), and take the Maximum Insolvency Value. 

### Estimate VaR by Total Estimated Debt

Above we saw how to estimate the maximum debt of one account.

Now, we want to estimate the Value at Risk of Compound. We do the following:
- For `mc_paths` (N) simulations of the asset prices.
- Calculate the potential debt of all `var_accounts`. 
- Sum the debt if the account turns unhealthy (collateral < borrow), call `total_debt`
- Now have N total debts. Find the `var_percentile` value as the `VaR` value 

> There is a 10% chance Compound will have a loss of 443899 USD or more, due to under-collateralized accounts. The simulation is based of a time horizon of 100 days. And a sample of 1000 accounts.

When requesting account info, I limited to accounts with `min_borrow_value_in_eth = 0.2` and `max_health = 1.5` so they likely have some impact on the debt calculation (can be changed in `parameters.json`). The real loss will be bigger, because also seemingly more save accounts now could get into a dangerous state, also the small accounts with small borrow_value would add up. 

### Runtime
The use of numpy and matrix multiplication keep the runtime low. On my machine (Intel MacBook Pro), the Monte Carlo Simulation of 10.000 * 17 paths for 100 days horizon only took 10 sec. 
Also the `calc_supply/borrow_value_usd` functions use matrix multiplication and return instantly (output 10.000 * 17).

Requesting the account information from the `CompoundAPI` can take a while. In the current setup with `var_accounts=1000` it took me 60 seconds. The `get_accounts()` request limits the request to max 200 accounts each page, and sleeps for a few seconds between requests. 

What takes time is to download the historical prices, this is why the download code is excluded and here we load the prices from a csv.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

from src.the_graph_api import TheGraphAPI

PRICES_FILE = Path.cwd().joinpath("data", "coingecko_prices.csv")
PARAMS_FILE = Path.cwd().joinpath("configs", "parameters.json")


In [2]:
# get current Compound Borrows from The Graph

tg_api = TheGraphAPI()
borrows = tg_api.get_compound_borrows()

col_dtypes = {
    "symbol":str,
    "underlyingSymbol":str,
    "underlyingName":str, 
    "totalBorrows":float,	
    "totalSupply":float,
    "collateralFactor":float,	
    "exchangeRate":float,
    "underlyingPriceUSD":float,	
    "underlyingPrice":float,
    "blockTimestamp":int,
    }
borrows_df = pd.DataFrame(borrows)
borrows_df = borrows_df.apply(lambda c: c.astype(col_dtypes[c.name]))
borrows_df

Have two entries for `DAI` and `wBTC` each... only keep the bigger one for now
```
{
  "underlyingSymbol": "DAI",
  "underlyingName": "Dai Stablecoin",
  "borrowRate": "0.0233026004452608",
  "totalBorrows": "211524161.923877439677332104",
  "collateralFactor": "0.835",
  "underlyingPriceUSD": "1.000773",
  "blockTimestamp": 1669045535,
  "accrualBlockNumber": 16019369
},
{
  "underlyingSymbol": "DAI",
  "underlyingName": "Dai Stablecoin v1.0 (DAI)",
  "borrowRate": "0.0501467542606656",
  "totalBorrows": "240.64852988575940222",
  "collateralFactor": "0",
  "underlyingPriceUSD": "6.210826",
  "blockTimestamp": 1668979619,
  "accrualBlockNumber": 16013904
},
....
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0206864728473312",
  "totalBorrows": "0.39387562",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "15929.996902",
  "blockTimestamp": 1669001867,
  "accrualBlockNumber": 16015751
},
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0221761103560704",
  "totalBorrows": "152.21575285",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "16182.771321",
  "blockTimestamp": 1669043411,
  "accrualBlockNumber": 16019192
},
```

In [ ]:
borrows_df = borrows_df.sort_values(by=["underlyingSymbol", "totalBorrows"], ascending=False).reset_index(drop=True)
borrows_df = borrows_df.drop_duplicates(subset=["underlyingSymbol"])
borrows_df = borrows_df.sort_values(by=["underlyingSymbol"]).reset_index(drop=True)
borrows_df

### Monte Carlo Simulation
- Calculate Volatilties
- Simulate Price Paths

In [ ]:
from src.measures import calc_volatility, monte_carlo_simulation

with open(PARAMS_FILE) as f:
    params = json.load(f)
    
# load historical prices from coingecko (used scrip `download_historica_prices.ipynb` to prepare data)
prices = pd.read_csv(PRICES_FILE)
symbols = sorted(prices["symbol"].unique())

# calculate volatilities
vols = {}
for symbol in symbols:
    p = prices[prices["symbol"] == symbol]["price"].values
    vols[symbol] = calc_volatility(p)

print(vols)

In [ ]:
# plot example of Monte Carlo Paths (aave)
from src.measures import simulate_path
from src.plots import plot_mc_paths
from matplotlib import pyplot as plt

symbol = "aave"
price = borrows_df[borrows_df["underlyingSymbol"] == symbol.upper()]["underlyingPriceUSD"].iloc[0]
paths = [simulate_path(price, vols[symbol], horizon=20) for _ in range(10)]

fig = plot_mc_paths(symbol, paths)
plt.show()

In [ ]:
# Simulate Monte Carlo Paths for all Symbols
mc_paths = {}
for symbol in symbols:
    price = borrows_df[borrows_df["underlyingSymbol"] == symbol.upper()]["underlyingPriceUSD"].iloc[0]
    mc_paths[symbol] = monte_carlo_simulation(price, vols[symbol], params["mc_horizon"], params["mc_paths"])

mc_prices = pd.DataFrame(mc_paths).T
mc_prices_small = mc_prices[mc_prices.columns.tolist()[:10]]

print(f"For each asset simulated {params['mc_paths']} paths with time horizon of {params['mc_horizon']} days.")
print(f"Showing 10 example states:")
mc_prices_small

# Calculate Value of Borrows and Collateral

- value of cTokens in USD: `amount_cToken * exchange_rate * price_usd`
- calculate `collateralValue` and `borrowValue` 

The `health` on Compound is `collateralValue / borrowValue`. A value of < 1 means, that the account is under-collateralized and subject to liquidation. The `health` value from the Compound API is calculated with the collateral and borrow values in ETH (`total_collateral_value_in_eth / total_borrow_value_in_eth.`). Since my simulation is based on the historic asset prices in USD, I will also calculate the health based on collateral and borrow value in USD. 

## Maximum Insolvency for an Account

To estimate the maximum Insolvency, we use the 10 simulated szenarios of assets prices. 

`borrowValue` for each asset is calculated using the field `borrow_balance_underlying` of the account. 

`collateralValue` is the sum of `supply_balance_underlying` multiplied by their `collatoralFactor`s. 

Then the Insolvency is `insolvency = borrowValue - collateralValue`. We do this 10 times (for the 10 simulated szenarios), and take the Maximum Insolvency Value. 

In [ ]:
# get accounts from compound that are already close to insolvency (max_health = 1.5)

from src.compound_api import CompoundApi
c_api = CompoundApi()
accounts = c_api.get_accounts(amount=4, max_health=1.5, min_borrow_value_in_eth=0.2)

In [ ]:
a = accounts[1]
print(a["total_borrow_value_in_eth"], a["total_collateral_value_in_eth"], a["health"])
pd.DataFrame(a["tokens"])

In [ ]:
# calculate account's health using the usd value

def format_assets_to_df(tokens, field):
    empty_tokens_df = pd.DataFrame(index=symbols)
    tokens = {t["symbol"][1:].lower(): float(t[field]["value"]) for t in tokens}
    df = pd.Series(tokens)

    empty_tokens_df["amount"] = df
    empty_tokens_df.fillna(0, inplace=True)
    return empty_tokens_df["amount"].values

account_tokens = pd.DataFrame(index=symbols)
account_tokens["borrow"] = format_assets_to_df(a["tokens"], "borrow_balance_underlying")
account_tokens["supply"]  = format_assets_to_df(a["tokens"], "supply_balance_underlying")
account_tokens

In [ ]:
from src.measures import calc_borrow_value_usd, calc_supply_value_usd

b = calc_borrow_value_usd(
    n_ctoken=account_tokens["borrow"].values, 
    exchange_rate=borrows_df["exchangeRate"].values, 
    prices=borrows_df["underlyingPriceUSD"].values
)

c = calc_supply_value_usd(
    n_ctoken=account_tokens["supply"].values, 
    exchange_rate=borrows_df["exchangeRate"].values, 
    collateral_factor=borrows_df["collateralFactor"].values,
    prices=borrows_df["underlyingPriceUSD"].values
)
print(f"calculated health of account (usd price): {c.sum()/b.sum()}, api health (eth price) {a['health']['value'][:5]}.")

### How would the Monte Carlo simulated prices affect the supply and borrow vlaues?

In [ ]:
mc_borrow_values = calc_borrow_value_usd(
    n_ctoken=account_tokens["borrow"].values, 
    exchange_rate=borrows_df["exchangeRate"].values, 
    prices=mc_prices_small.values
)
mc_supply_values = calc_supply_value_usd(
    n_ctoken=account_tokens["supply"].values, 
    exchange_rate=borrows_df["exchangeRate"].values, 
    collateral_factor=borrows_df["collateralFactor"].values,
    prices = mc_prices_small.values
)

# calculate simulated debts
debt = mc_borrow_values.sum(axis=0) - mc_supply_values.sum(axis=0)
max_debt = debt.max()
print(f"Max debt of account {a['address']} from {len(mc_prices_small.columns)} simulated szenarios: {max_debt} USD")

### Estimate VaR by Total Estimated Debt

Above we saw how to estimate the maximum debt of one account.

Now, we want to estimate the Value at Risk of Compound. We do the following:
- For `mc_paths` (N) simulations of the asset prices
- Calculate the potential debt of all `var_accounts`. 
- Sum the debt if the account turns unhealthy (collateral < borrow), call `total_debt`
- Now have N total debts. Find the `var_percentile` value as the `VaR` value 

>There is a 10% chance that Compound will have a loss of `VaR`, due to insolvent accounts. 

Limit to accounts with `min_borrow_value_in_eth = 0.2` so they have some impact (can be changed in `parameters.json`)


In [ ]:
# if `var_accounts` is big, this could run a while (1000 accounts took for me 1 minute.)

accounts = c_api.get_accounts(
    amount=params["var_accounts"], 
    min_borrow_value_in_eth=params["accounts_min_borrow_eth"], 
    max_health=params["accounts_max_health"]
    )

In [ ]:
total_debt = np.zeros(params["mc_paths"])
always_healthy_accounts = 0
for a in accounts:
    mc_borrow_values = calc_borrow_value_usd(
        n_ctoken=format_assets_to_df(a["tokens"], "borrow_balance_underlying"), 
        exchange_rate=borrows_df["exchangeRate"].values, 
        prices=mc_prices.values
    )
    mc_supply_values = calc_supply_value_usd(
        n_ctoken=format_assets_to_df(a["tokens"], "supply_balance_underlying"), 
        exchange_rate=borrows_df["exchangeRate"].values, 
        collateral_factor=borrows_df["collateralFactor"].values,
        prices = mc_prices.values
    )

    # calculate simulated debts
    debt = mc_borrow_values.sum(axis=0) - mc_supply_values.sum(axis=0)
    # only keep positive debts ("account is unhealthy")
    debt = np.maximum(np.zeros(params["mc_paths"]), debt)

    total_debt += debt
    if debt.sum()==0:
        always_healthy_accounts += 1

print(f"Of {params['var_accounts']}, {always_healthy_accounts} always stayed healthy, " \
    f"in all {params['mc_paths']} simulations. " \
    "If too many accounts stay health, change parameters `accounts_min_borrow_eth` or `accounts_max_health` for the `get_accounts` request.")


In [ ]:
from src.plots import plot_loss_histogram

fig = plot_loss_histogram(total_debt, params["mc_paths"])

var_critical_debt = np.percentile(total_debt, 1-params["var_percentile"])
print(f"There is a {int(params['var_percentile']*100)}% chance Compound will have a loss of {var_critical_debt:.0f} USD or more, due to under-collateralized accounts. "\
    f"The simulation is based of a time horizon of {params['mc_horizon']} days. And a sample of {params['var_accounts']} accounts.")